In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import nltk.classify.util
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import sentiwordnet as swn,SentiSynset
import numpy as np
import re
import string
import nltk
%matplotlib inline

In [2]:
#data=pd.read_csv("C:/Users/roopeshn/Desktop/sample.csv")
data=pd.read_csv("amazon_reviews_us_Gift_Card_v1_00.tsv",sep="\t",error_bad_lines=False)
data.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,24371595,R27ZP1F1CD0C3Y,B004LLIL5A,346014806,Amazon eGift Card - Celebrate,Gift Card,5,0,0,N,Y,Five Stars,Great birthday gift for a young adult.,2015-08-31
1,US,42489718,RJ7RSBCHUDNNE,B004LLIKVU,473048287,Amazon.com eGift Cards,Gift Card,5,0,0,N,Y,Gift card for the greatest selection of items ...,It's an Amazon gift card and with over 9823983...,2015-08-31
2,US,861463,R1HVYBSKLQJI5S,B00IX1I3G6,926539283,Amazon.com Gift Card Balance Reload,Gift Card,5,0,0,N,Y,Five Stars,Good,2015-08-31
3,US,25283295,R2HAXF0IIYQBIR,B00IX1I3G6,926539283,Amazon.com Gift Card Balance Reload,Gift Card,1,0,0,N,Y,One Star,Fair,2015-08-31
4,US,397970,RNYLPX611NB7Q,B005ESMGV4,379368939,"Amazon.com Gift Cards, Pack of 3 (Various Desi...",Gift Card,5,0,0,N,Y,Five Stars,I can't believe how quickly Amazon can get the...,2015-08-31


In [3]:
permanent = data[['star_rating' , 'review_headline' , 'review_body' ]]
print(permanent.isnull().sum()) #Checking for null values
permanent.head()

star_rating        0
review_headline    0
review_body        0
dtype: int64


,star_rating,review_headline,review_body
0,5,Five Stars,Great birthday gift for a young adult.
1,5,Gift card for the greatest selection of items ...,It's an Amazon gift card and with over 9823983...
2,5,Five Stars,Good
3,1,One Star,Fair
4,5,Five Stars,I can't believe how quickly Amazon can get the...


In [4]:
check =  permanent[permanent["star_rating"].isnull()]
check.head()

,star_rating,review_headline,review_body


In [5]:
check =  permanent[permanent["star_rating"].notnull()]
check.head()

,star_rating,review_headline,review_body
0,5,Five Stars,Great birthday gift for a young adult.
1,5,Gift card for the greatest selection of items ...,It's an Amazon gift card and with over 9823983...
2,5,Five Stars,Good
3,1,One Star,Fair
4,5,Five Stars,I can't believe how quickly Amazon can get the...


In [6]:
review=pd.DataFrame(data.groupby('star_rating').size().sort_values(ascending=False).rename('No of Users').reset_index())
review.head()

,star_rating,No of Users
0,5,1266
1,4,85
2,1,54
3,3,33
4,2,14


In [7]:
actualrating = permanent[(permanent['star_rating'] == 1) | (permanent['star_rating'] == 5)]
actualrating.shape

(1320, 3)

In [8]:
y = actualrating['star_rating']
x = actualrating['review_body'].reset_index()

In [9]:
len(y)
#len(x)

1320

In [10]:
X = x['review_body']
Y=X[pd.notnull(X)]
print(Y)

0                  Great birthday gift for a young adult.
1       It's an Amazon gift card and with over 9823983...
2                                                    Good
3                                                    Fair
4       I can't believe how quickly Amazon can get the...
5       Perfect!  Nice & easy plus everyone loves them...
6                                                excelent
7                              Great and Safe Gift Giving
8                                           What?????????
9                   This was just too easy~click & print!
10                                                   Bien
11      Easy to print from email.  I love the variety ...
12                                 Amazing with 10 dollar
13                                  Remember Matthew 7:21
14                                                   good
15                      Awesome way to send a gift card!!
16      I love that I have instant, helpful options wh...
17            

In [11]:
import string
from nltk.corpus import stopwords
# stop=set(stopwords.words('english'))
def text_process(text):
    '''
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Return the cleaned text as a list of words
    '''
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [12]:
tokens = Y[0].split()
print(tokens)

['Great', 'birthday', 'gift', 'for', 'a', 'young', 'adult.']


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
# next we need to vectorize our input variable (X)
#we use the count vectoriser function and the analyser we use is the above lines of code
# this should return a vector array
bow_transformer = CountVectorizer(analyzer=text_process).fit(Y)
len(bow_transformer.vocabulary_)

1973

In [14]:
#review_24 = Y[24]

In [15]:
#bow_25 = bow_transformer.transform([review_24])
#bow_25

In [16]:
#print(bow_25)

In [17]:
Y = bow_transformer.transform(Y)

In [18]:
from sklearn.model_selection import train_test_split
#using 30% of the data for testing, this will be revised once we do not get the desired accuracy
Y_train, Y_test, y_train, y_test = train_test_split(Y, y, test_size=0.3, random_state=101)

# Naive Bayes Classifier


In [19]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(Y_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [20]:
preds = nb.predict(Y_test)

In [21]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, preds))
print('\n')
print(classification_report(y_test, preds))
nb.score(Y_train, y_train)

[[  2   7]
 [  5 382]]


              precision    recall  f1-score   support

           1       0.29      0.22      0.25         9
           5       0.98      0.99      0.98       387

    accuracy                           0.97       396
   macro avg       0.63      0.60      0.62       396
weighted avg       0.97      0.97      0.97       396



0.9805194805194806